#Asignatura: Visión por Computador

##Alumnos: Yeray Álvarez-Buylla Parra, María Elena Navarro Santana

##Trabajo final

###Paquetes necesarios

In [31]:
import torch
from torchvision.models.detection import retinanet_resnet50_fpn
from torchvision.transforms import functional as F
from torch.utils.data import DataLoader, Dataset
import cv2
import os
import json
from tqdm import tqdm  
from torch.cuda.amp import GradScaler, autocast  
from torchvision.ops import sigmoid_focal_loss  
import math
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from ultralytics import YOLO
import pandas as pd

###Conjunto de datos

In [35]:
# Directorios del modelo 1
base_dir = 'C:/Users/lenin/Documents/Universidad_2024-2025/VC/Trabajo_final/Pruebas/Bone-Coco'
train_dir = base_dir + '/train'
val_dir = base_dir + '/valid'
test_dir = base_dir + '/test'

# Directorios del modelo 2
base_dir2 = 'C:/Users/lenin/Documents/Universidad_2024-2025/VC/Trabajo_final/Pruebas2/bone2_coco'
train_dir2 = base_dir2 + '/train'
val_dir2 = base_dir2 + '/valid'
test_dir2 = base_dir2 + '/test'

# Directorios del modelo 3
base_dir3 = 'C:/Users/lenin/Documents/Universidad_2024-2025/VC/Trabajo_final/Pruebas3/bone3_coco'
train_dir3 = base_dir3 + '/train'
val_dir3 = base_dir3 + '/valid'
test_dir3 = base_dir3 + '/test'

In [36]:
def count_images(directory, extensions=("jpg", "jpeg", "png")):
    # Filtrar archivos solo por las extensiones de imágenes
    return len([f for f in os.listdir(directory) 
                if os.path.isfile(os.path.join(directory, f)) and f.lower().endswith(extensions)])

# Contar imágenes en cada conjunto
train_count = count_images(train_dir)
val_count = count_images(val_dir)
test_count = count_images(test_dir)

print("Tamaño del conjunto de entrenamiento modelo 1:", train_count)
print("Tamaño del conjunto de validación modelo 1:", val_count)
print("Tamaño del conjunto de test modelo 1:", test_count)

# Contar imágenes en cada conjunto
train_count2 = count_images(train_dir2)
val_count2 = count_images(val_dir2)
test_count2 = count_images(test_dir2)

print("Tamaño del conjunto de entrenamiento modelo 2:", train_count2)
print("Tamaño del conjunto de validación modelo 2:", val_count2)
print("Tamaño del conjunto de test modelo 2:", test_count2)

# Contar imágenes en cada conjunto
train_count3 = count_images(train_dir3)
val_count3 = count_images(val_dir3)
test_count3 = count_images(test_dir3)

print("Tamaño del conjunto de entrenamiento modelo 3:", train_count3)
print("Tamaño del conjunto de validación modelo 3:", val_count3)
print("Tamaño del conjunto de test modelo 3:", test_count3)

Tamaño del conjunto de entrenamiento modelo 1: 1411
Tamaño del conjunto de validación modelo 1: 403
Tamaño del conjunto de test modelo 1: 201
Tamaño del conjunto de entrenamiento modelo 2: 831
Tamaño del conjunto de validación modelo 2: 114
Tamaño del conjunto de test modelo 2: 49
Tamaño del conjunto de entrenamiento modelo 3: 1742
Tamaño del conjunto de validación modelo 3: 494
Tamaño del conjunto de test modelo 3: 264


###YOLO

In [39]:
# Define rutas de los modelos y del conjunto de pruebas del dataset 1
models = [
    {
        "name": "Model 1",
        "model_path": r"C:/Users/lenin/Documents/Universidad_2024-2025/VC/Trabajo_final/Pruebas/Bone-Yolov11/runs/detect/train/weights/best.pt",
    },
    {
        "name": "Model 2",
        "model_path": r"C:/Users/lenin/Documents/Universidad_2024-2025/VC/Trabajo_final/Pruebas2/bone2_yolov11/runs/detect/train/weights/best.pt",
    },
    {
        "name": "Model 3",
        "model_path": r"C:/Users/lenin/Documents/Universidad_2024-2025/VC/Trabajo_final/Pruebas3/bone3_yolov11/runs/detect/train/weights/best.pt",
    },
]

# Ruta al archivo YAML que define el conjunto de pruebas 1
common_test_yaml = r"C:/Users/lenin/Documents/Universidad_2024-2025/VC/Trabajo_final/Pruebas/Bone-Yolov11/data/miarchivo.yml"

# Evalua cada modelo en el mismo conjunto de pruebas
results = []

for model_info in models:
    model = YOLO(model_info["model_path"])  # Carga el modelo
    metrics = model.val(data=common_test_yaml)  # Evaluar en el conjunto de pruebas 1
    results.append({
        "model": model_info["name"],
        "precision": metrics.box.p,
        "recall": metrics.box.r,
        "mAP50": metrics.box.map50,
        "mAP50-95": metrics.box.map,
    })

# Muestra y guardar los resultados
df = pd.DataFrame(results)
df.sort_values(by="mAP50-95", ascending=False, inplace=True)

print(df)  # Mostrar en consola
df.to_csv("comparacion_modelo_1.csv", index=False)  # Guarda en un archivo CSV

Ultralytics 8.3.40  Python-3.9.21 torch-2.6.0.dev20241230+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning C:\Users\lenin\Documents\Universidad_2024-2025\VC\Trabajo_final\Pruebas\Bone-Yolov11\valid\labels.cache... 403 images, 2 backgrounds, 0 corrupt: 100%|██████████| 403/403 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:03<00:00,  8.34it/s]


                   all        403        661      0.659      0.635       0.65      0.267
Speed: 0.6ms preprocess, 3.1ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs\detect\val2
Ultralytics 8.3.40  Python-3.9.21 torch-2.6.0.dev20241230+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning C:\Users\lenin\Documents\Universidad_2024-2025\VC\Trabajo_final\Pruebas\Bone-Yolov11\valid\labels.cache... 403 images, 2 backgrounds, 0 corrupt: 100%|██████████| 403/403 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:03<00:00,  8.47it/s]


                   all        403        661      0.225      0.139      0.124     0.0348
Speed: 0.5ms preprocess, 3.3ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs\detect\val3
Ultralytics 8.3.40  Python-3.9.21 torch-2.6.0.dev20241230+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning C:\Users\lenin\Documents\Universidad_2024-2025\VC\Trabajo_final\Pruebas\Bone-Yolov11\valid\labels.cache... 403 images, 2 backgrounds, 0 corrupt: 100%|██████████| 403/403 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:02<00:00,  9.20it/s]


                   all        403        661      0.054     0.0454     0.0141    0.00299
Speed: 0.6ms preprocess, 3.1ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect\val4
     model              precision                 recall     mAP50  mAP50-95
0  Model 1   [0.6591128625515691]   [0.6347575049062814]  0.649996  0.267395
1  Model 2  [0.22503586942857806]  [0.13918305597579425]  0.123874  0.034837
2  Model 3  [0.05395901261848612]   [0.0453857791225416]  0.014051  0.002993


In [40]:
# Define rutas de los modelos y del conjunto de pruebas del dataset 1
models = [
    {
        "name": "Model 1",
        "model_path": r"C:/Users/lenin/Documents/Universidad_2024-2025/VC/Trabajo_final/Pruebas/Bone-Yolov11/runs/detect/train/weights/best.pt",
    },
    {
        "name": "Model 2",
        "model_path": r"C:/Users/lenin/Documents/Universidad_2024-2025/VC/Trabajo_final/Pruebas2/bone2_yolov11/runs/detect/train/weights/best.pt",
    },
    {
        "name": "Model 3",
        "model_path": r"C:/Users/lenin/Documents/Universidad_2024-2025/VC/Trabajo_final/Pruebas3/bone3_yolov11/runs/detect/train/weights/best.pt",
    },
]

# Ruta al archivo YAML que define el conjunto de pruebas 1
common_test_yaml = r"C:/Users/lenin/Documents/Universidad_2024-2025/VC/Trabajo_final/Pruebas2/bone2_yolov11/data/miarchivo.yml"

# Evalua cada modelo en el mismo conjunto de pruebas
results = []

for model_info in models:
    model = YOLO(model_info["model_path"])  # Carga el modelo
    metrics = model.val(data=common_test_yaml)  # Evaluar en el conjunto de pruebas 1
    results.append({
        "model": model_info["name"],
        "precision": metrics.box.p,
        "recall": metrics.box.r,
        "mAP50": metrics.box.map50,
        "mAP50-95": metrics.box.map,
    })

# Muestra y guardar los resultados
df = pd.DataFrame(results)
df.sort_values(by="mAP50-95", ascending=False, inplace=True)

print(df)  # Mostrar en consola
df.to_csv("comparacion_modelo_2.csv", index=False)  # Guarda en un archivo CSV

Ultralytics 8.3.40  Python-3.9.21 torch-2.6.0.dev20241230+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning C:\Users\lenin\Documents\Universidad_2024-2025\VC\Trabajo_final\Pruebas2\bone2_yolov11\valid\labels.cache... 114 images, 0 backgrounds, 0 corrupt: 100%|██████████| 114/114 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:01<00:00,  5.71it/s]


                   all        114        120     0.0968       0.15     0.0402      0.015
Speed: 1.6ms preprocess, 4.2ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs\detect\val5
Ultralytics 8.3.40  Python-3.9.21 torch-2.6.0.dev20241230+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning C:\Users\lenin\Documents\Universidad_2024-2025\VC\Trabajo_final\Pruebas2\bone2_yolov11\valid\labels.cache... 114 images, 0 backgrounds, 0 corrupt: 100%|██████████| 114/114 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:01<00:00,  4.61it/s]


                   all        114        120      0.527      0.352      0.386      0.141
Speed: 1.7ms preprocess, 4.6ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs\detect\val6
Ultralytics 8.3.40  Python-3.9.21 torch-2.6.0.dev20241230+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning C:\Users\lenin\Documents\Universidad_2024-2025\VC\Trabajo_final\Pruebas2\bone2_yolov11\valid\labels.cache... 114 images, 0 backgrounds, 0 corrupt: 100%|██████████| 114/114 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:01<00:00,  7.69it/s]


                   all        114        120     0.0918     0.0667     0.0276    0.00695
Speed: 0.7ms preprocess, 2.7ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs\detect\val7
     model              precision                 recall     mAP50  mAP50-95
1  Model 2   [0.5266365520346729]   [0.3523256265755877]  0.385535  0.140828
0  Model 1  [0.09684600658739738]                 [0.15]  0.040196  0.014970
2  Model 3  [0.09182698936346073]  [0.06666666666666667]  0.027572  0.006949


In [41]:
# Define rutas de los modelos y del conjunto de pruebas del dataset 1
models = [
    {
        "name": "Model 1",
        "model_path": r"C:/Users/lenin/Documents/Universidad_2024-2025/VC/Trabajo_final/Pruebas/Bone-Yolov11/runs/detect/train/weights/best.pt",
    },
    {
        "name": "Model 2",
        "model_path": r"C:/Users/lenin/Documents/Universidad_2024-2025/VC/Trabajo_final/Pruebas2/bone2_yolov11/runs/detect/train/weights/best.pt",
    },
    {
        "name": "Model 3",
        "model_path": r"C:/Users/lenin/Documents/Universidad_2024-2025/VC/Trabajo_final/Pruebas3/bone3_yolov11/runs/detect/train/weights/best.pt",
    },
]

# Ruta al archivo YAML que define el conjunto de pruebas 1
common_test_yaml = r"C:/Users/lenin/Documents/Universidad_2024-2025/VC/Trabajo_final/Pruebas3/bone3_yolov11/data/miarchivo.yml"

# Evalua cada modelo en el mismo conjunto de pruebas
results = []

for model_info in models:
    model = YOLO(model_info["model_path"])  # Carga el modelo
    metrics = model.val(data=common_test_yaml)  # Evaluar en el conjunto de pruebas 1
    results.append({
        "model": model_info["name"],
        "precision": metrics.box.p,
        "recall": metrics.box.r,
        "mAP50": metrics.box.map50,
        "mAP50-95": metrics.box.map,
    })

# Muestra y guardar los resultados
df = pd.DataFrame(results)
df.sort_values(by="mAP50-95", ascending=False, inplace=True)

print(df)  # Mostrar en consola
df.to_csv("comparacion_modelo_3.csv", index=False)  # Guarda en un archivo CSV

Ultralytics 8.3.40  Python-3.9.21 torch-2.6.0.dev20241230+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning C:\Users\lenin\Documents\Universidad_2024-2025\VC\Trabajo_final\Pruebas3\bone3_yolov11\valid\labels.cache... 494 images, 2 backgrounds, 0 corrupt: 100%|██████████| 494/494 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 31/31 [00:03<00:00,  8.83it/s]


                   all        494        560     0.0186     0.0196      0.003   0.000621
Speed: 0.4ms preprocess, 2.8ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs\detect\val8
Ultralytics 8.3.40  Python-3.9.21 torch-2.6.0.dev20241230+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning C:\Users\lenin\Documents\Universidad_2024-2025\VC\Trabajo_final\Pruebas3\bone3_yolov11\valid\labels.cache... 494 images, 2 backgrounds, 0 corrupt: 100%|██████████| 494/494 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 31/31 [00:03<00:00,  8.56it/s]


                   all        494        560      0.142      0.111     0.0458    0.00996
Speed: 0.3ms preprocess, 3.0ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs\detect\val9
Ultralytics 8.3.40  Python-3.9.21 torch-2.6.0.dev20241230+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning C:\Users\lenin\Documents\Universidad_2024-2025\VC\Trabajo_final\Pruebas3\bone3_yolov11\valid\labels.cache... 494 images, 2 backgrounds, 0 corrupt: 100%|██████████| 494/494 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 31/31 [00:03<00:00,  8.67it/s]


                   all        494        560      0.901      0.821      0.886      0.459
Speed: 0.4ms preprocess, 3.2ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect\val10
     model               precision                  recall     mAP50  mAP50-95
2  Model 3     [0.901140241906021]    [0.8214285714285714]  0.886305  0.459133
1  Model 2    [0.1423219414076907]   [0.11071428571428571]  0.045844  0.009958
0  Model 1  [0.018614454595773443]  [0.019642857142857142]  0.003003  0.000621


El mejor modelo para su conjunto de datos es el modelo 3 debido a sus métricas batantes altas respecto a otros modelos.

###RetinaNet

In [45]:
# 1. Crea el Dataset personalizado
class FractureDataset(Dataset):
    def __init__(self, images_path, annotations_file, transforms=None):
        self.images_path = images_path
        with open(annotations_file, 'r') as f:
            data = json.load(f)
        
        self.images_info = {img['id']: img for img in data['images']}
        self.annotations = self.process_annotations(data['annotations'])
        self.transforms = transforms

    def process_annotations(self, annotations):
        processed = {}
        for ann in annotations:
            img_id = ann['image_id']
            if img_id not in processed:
                processed[img_id] = {'boxes': [], 'labels': []}
            bbox = ann['bbox']
            x_min, y_min, width, height = bbox
            x_max = x_min + width
            y_max = y_min + height
            processed[img_id]['boxes'].append([x_min, y_min, x_max, y_max])
            processed[img_id]['labels'].append(1)  # Clase "fracture" siempre es 1
        return processed

    def __len__(self):
        return len(self.images_info)

    def __getitem__(self, idx):
        img_id = list(self.images_info.keys())[idx]
        img_info = self.images_info[img_id]
        img_path = os.path.join(self.images_path, img_info['file_name'])
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Obtiene las anotaciones
        if img_id in self.annotations:
            annotations = self.annotations[img_id]
            boxes = torch.tensor(annotations['boxes'], dtype=torch.float32)
            labels = torch.tensor(annotations['labels'], dtype=torch.int64)
        else:
            boxes = torch.empty((0, 4), dtype=torch.float32)
            labels = torch.empty((0,), dtype=torch.int64)

        target = {
            'boxes': boxes,
            'labels': labels,
            'image_id': torch.tensor([idx])
        }

        # Aplicar transformaciones
        if self.transforms:
            img = self.transforms(img)

        return img, target

# 2. Define transformaciones
class ResizeTransform:
    def __call__(self, image):
        image = cv2.resize(image, (640, 640)) 
        return F.to_tensor(image)

transforms = ResizeTransform()

# 3. Inicializa el dataset y dataloaders
images_path = r"C:\Users\lenin\Documents\Universidad_2024-2025\VC\Trabajo_final\Pruebas\Bone-Coco\train"
annotations_file = os.path.join(images_path, '_annotations.json')

dataset = FractureDataset(images_path, annotations_file, transforms=transforms)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

# 4. Carga RetinaNet preentrenado
model = retinanet_resnet50_fpn(pretrained=True)

num_classes = 2

out_channels = model.head.classification_head.conv[0].out_channels 
num_anchors = model.head.classification_head.num_anchors
model.head.classification_head.num_classes = num_classes

cls_logits = torch.nn.Conv2d(out_channels, num_anchors * num_classes, kernel_size=3, stride=1, padding=1)
torch.nn.init.normal_(cls_logits.weight, std=0.01)
torch.nn.init.constant_(cls_logits.bias, -math.log((1 - 0.01) / 0.01))

model.head.classification_head.cls_logits = cls_logits

# Congela todas las capas excepto la última
for param in model.parameters():
    param.requires_grad = False

# Activa gradiente solo para la última capa (cls_logits)
for param in model.head.classification_head.cls_logits.parameters():
    param.requires_grad = True

# 5. Configura el optimizador
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001, weight_decay=0.0001)
scaler = GradScaler()

# 6. Define la función de entrenamiento
def train_one_epoch(model, optimizer, dataloader, device, epoch, num_epochs):
    model.train()
    epoch_loss = 0

    with tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as pbar:
        for images, targets in pbar:
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            optimizer.zero_grad()

            try:
                with autocast():
                    loss_dict = model(images, targets)
                    if not loss_dict:
                        print("Lote vacío, saltando...")
                        continue

                    classification_logits = loss_dict.get('classification', torch.tensor(0.0, device=device))
                    bbox_regression = loss_dict.get('bbox_regression', torch.tensor(0.0, device=device))

                    # Verificar tensores vacíos
                    if classification_logits.numel() == 0 or bbox_regression.numel() == 0:
                        print("Tensor vacío en pérdidas, saltando lote...")
                        continue

                    # Cálculo de la pérdida con sigmoide focal
                    classification_loss = sigmoid_focal_loss(
                        classification_logits,
                        torch.zeros_like(classification_logits),
                        reduction="sum"
                    )

                    bbox_loss = bbox_regression.sum()

                    losses = classification_loss + bbox_loss

                    # Verificar valores numéricamente inestables
                    if not torch.isfinite(losses).all():
                        print("Pérdidas NaN o Inf, restableciendo gradientes y saltando lote...")
                        optimizer.zero_grad()
                        scaler.update()
                        continue

                # Backpropagación y optimización
                scaler.scale(losses).backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
                scaler.step(optimizer)
                scaler.update()

                epoch_loss += losses.item()
                pbar.set_postfix({"loss": f"{losses.item():.4f}"})

            except Exception as e:
                print(f"Error en el lote: {e}")
                continue

    return epoch_loss / len(dataloader)

# 7. Entrena el modelo
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# Cambiar a modo de evaluación para pruebas
def evaluate_model(model, dataloader, device):
    model.eval()  # Modo de evaluación
    results = []
    with torch.no_grad():  # Desactiva gradientes
        for images, targets in dataloader:
            images = [img.to(device) for img in images]
            outputs = model(images)
            results.extend(outputs)
    return results

num_epochs = 10
for epoch in range(num_epochs):
    epoch_loss = train_one_epoch(model, optimizer, dataloader, device, epoch, num_epochs)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")
    test_dataloader = DataLoader(dataset, batch_size=8, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))
    evaluation_results = evaluate_model(model, test_dataloader, device)

# 8. Guarda el modelo
torch.save(model.state_dict(), "retinanet_fracture.pth")
print("Modelo guardado en retinanet_fracture.pth")

C:\Users\lenin\AppData\Local\Temp\ipykernel_19560\2002529230.py:97: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
Epoch 1/10:   0%|          | 0/177 [00:00<?, ?batch/s]C:\Users\lenin\AppData\Local\Temp\ipykernel_19560\2002529230.py:112: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1/10: 100%|██████████| 177/177 [01:08<00:00,  2.58batch/s, loss=1.0213]


Epoch 1/10, Loss: 1.1221


Epoch 2/10: 100%|██████████| 177/177 [01:10<00:00,  2.52batch/s, loss=1.0706]


Epoch 2/10, Loss: 1.0937


Epoch 3/10: 100%|██████████| 177/177 [01:05<00:00,  2.70batch/s, loss=1.0709]


...

Epoch 8/10, Loss: 1.0512


Epoch 9/10: 100%|██████████| 177/177 [01:11<00:00,  2.47batch/s, loss=0.9694]


Epoch 9/10, Loss: 1.0573


Epoch 10/10: 100%|██████████| 177/177 [01:14<00:00,  2.39batch/s, loss=1.0974]


Epoch 10/10, Loss: 1.0602
Modelo guardado en retinanet_fracture.pth


In [46]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Carga el modelo entrenado
model_loaded = retinanet_resnet50_fpn(pretrained=True)

# Asegura que la capa cls_logits tiene las mismas dimensiones antes de cargar
model_loaded.head.classification_head.cls_logits = torch.nn.Conv2d(
    out_channels, num_anchors * num_classes, kernel_size=3, stride=1, padding=1
)
model_loaded.load_state_dict(torch.load("retinanet_fracture.pth", map_location=torch.device('cpu')))
model_loaded.eval()


# Prepara el conjunto de validación
images_path = r"C:\Users\lenin\Documents\Universidad_2024-2025\VC\Trabajo_final\Pruebas\Bone-Coco\valid"
annotations_file = os.path.join(images_path, '_annotations.json')

dataset_val = FractureDataset(images_path, annotations_file, transforms=transforms)
dataloader_val = DataLoader(dataset_val, batch_size=8, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

# Inicializa la métrica para calcular el mAP
metric = MeanAveragePrecision()

# Realiza la predicción y calcula las métricas
with torch.no_grad():  
    for images, targets in dataloader_val:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Realiza las predicciones
        predictions = model(images)

        # Actualiza la métrica con las predicciones y los valores reales
        metric.update(predictions, targets)

# Calcula las métricas
metrics = metric.compute()

# Almacena los resultados
results = []
results.append({
    "model": "RetinaNet",
    "precision": metrics["map_50"],  # mAP en IoU=0.5
    "recall": metrics["mar_100"],    # Recall con 100 detecciones
    "mAP50": metrics["map_50"],      # mAP en IoU=0.5
    "mAP50-95": metrics["map"]       # mAP en IoU=0.5:0.95
})

print(results)

c:\Users\lenin\anaconda3\envs\Trabajo_Final\lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: Encountered more than 100 detections in a single image. This means that certain detections with the lowest scores will be ignored, that may have an undesirable impact on performance. Please consider adjusting the `max_detection_threshold` to suit your use case. To disable this warning, set attribute class `warn_on_many_detections=False`, after initializing the metric.
  warnings.warn(*args, **kwargs)  # noqa: B028


[{'model': 'RetinaNet', 'precision': tensor(0.1091), 'recall': tensor(0.2584), 'mAP50': tensor(0.1091), 'mAP50-95': tensor(0.0265)}]


In [47]:
# 1. Crea el Dataset personalizado
class FractureDataset(Dataset):
    def __init__(self, images_path, annotations_file, transforms=None):
        self.images_path = images_path
        with open(annotations_file, 'r') as f:
            data = json.load(f)
        
        self.images_info = {img['id']: img for img in data['images']}
        self.annotations = self.process_annotations(data['annotations'])
        self.transforms = transforms

    def process_annotations(self, annotations):
        processed = {}
        for ann in annotations:
            img_id = ann['image_id']
            if img_id not in processed:
                processed[img_id] = {'boxes': [], 'labels': []}
            bbox = ann['bbox']
            x_min, y_min, width, height = bbox
            x_max = x_min + width
            y_max = y_min + height
            processed[img_id]['boxes'].append([x_min, y_min, x_max, y_max])
            processed[img_id]['labels'].append(1)  # Clase "fracture" siempre es 1
        return processed

    def __len__(self):
        return len(self.images_info)

    def __getitem__(self, idx):
        img_id = list(self.images_info.keys())[idx]
        img_info = self.images_info[img_id]
        img_path = os.path.join(self.images_path, img_info['file_name'])
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Obtiene las anotaciones
        if img_id in self.annotations:
            annotations = self.annotations[img_id]
            boxes = torch.tensor(annotations['boxes'], dtype=torch.float32)
            labels = torch.tensor(annotations['labels'], dtype=torch.int64)
        else:
            boxes = torch.empty((0, 4), dtype=torch.float32)
            labels = torch.empty((0,), dtype=torch.int64)

        target = {
            'boxes': boxes,
            'labels': labels,
            'image_id': torch.tensor([idx])
        }

        # Aplicar transformaciones
        if self.transforms:
            img = self.transforms(img)

        return img, target

# 2. Define transformaciones
class ResizeTransform:
    def __call__(self, image):
        image = cv2.resize(image, (640, 640)) 
        return F.to_tensor(image)

transforms = ResizeTransform()

# 3. Inicializa el dataset y dataloaders
images_path = r"C:\Users\lenin\Documents\Universidad_2024-2025\VC\Trabajo_final\Pruebas2\bone2_coco\train"
annotations_file = os.path.join(images_path, '_annotations.json')

dataset = FractureDataset(images_path, annotations_file, transforms=transforms)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

# 4. Carga RetinaNet preentrenado
model = retinanet_resnet50_fpn(pretrained=True)

num_classes = 2

out_channels = model.head.classification_head.conv[0].out_channels 
num_anchors = model.head.classification_head.num_anchors
model.head.classification_head.num_classes = num_classes

cls_logits = torch.nn.Conv2d(out_channels, num_anchors * num_classes, kernel_size=3, stride=1, padding=1)
torch.nn.init.normal_(cls_logits.weight, std=0.01)
torch.nn.init.constant_(cls_logits.bias, -math.log((1 - 0.01) / 0.01))

model.head.classification_head.cls_logits = cls_logits

# Congela todas las capas excepto la última
for param in model.parameters():
    param.requires_grad = False

# Activa gradiente solo para la última capa (cls_logits)
for param in model.head.classification_head.cls_logits.parameters():
    param.requires_grad = True

# 5. Configura el optimizador
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001, weight_decay=0.0001)
scaler = GradScaler()

# 6. Define la función de entrenamiento
def train_one_epoch(model, optimizer, dataloader, device, epoch, num_epochs):
    model.train()
    epoch_loss = 0

    with tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as pbar:
        for images, targets in pbar:
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            optimizer.zero_grad()

            try:
                with autocast():
                    loss_dict = model(images, targets)
                    if not loss_dict:
                        print("Lote vacío, saltando...")
                        continue

                    classification_logits = loss_dict.get('classification', torch.tensor(0.0, device=device))
                    bbox_regression = loss_dict.get('bbox_regression', torch.tensor(0.0, device=device))

                    # Verificar tensores vacíos
                    if classification_logits.numel() == 0 or bbox_regression.numel() == 0:
                        print("Tensor vacío en pérdidas, saltando lote...")
                        continue

                    # Cálculo de la pérdida con sigmoide focal
                    classification_loss = sigmoid_focal_loss(
                        classification_logits,
                        torch.zeros_like(classification_logits),
                        reduction="sum"
                    )

                    bbox_loss = bbox_regression.sum()

                    losses = classification_loss + bbox_loss

                    # Verificar valores numéricamente inestables
                    if not torch.isfinite(losses).all():
                        print("Pérdidas NaN o Inf, restableciendo gradientes y saltando lote...")
                        optimizer.zero_grad()
                        scaler.update()
                        continue

                # Backpropagación y optimización
                scaler.scale(losses).backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
                scaler.step(optimizer)
                scaler.update()

                epoch_loss += losses.item()
                pbar.set_postfix({"loss": f"{losses.item():.4f}"})

            except Exception as e:
                print(f"Error en el lote: {e}")
                continue

    return epoch_loss / len(dataloader)

# 7. Entrena el modelo
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# Cambiar a modo de evaluación para pruebas
def evaluate_model(model, dataloader, device):
    model.eval()  # Modo de evaluación
    results = []
    with torch.no_grad():  # Desactiva gradientes
        for images, targets in dataloader:
            images = [img.to(device) for img in images]
            outputs = model(images)
            results.extend(outputs)
    return results

num_epochs = 10
for epoch in range(num_epochs):
    epoch_loss = train_one_epoch(model, optimizer, dataloader, device, epoch, num_epochs)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")
    test_dataloader = DataLoader(dataset, batch_size=8, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))
    evaluation_results = evaluate_model(model, test_dataloader, device)

# 8. Guarda el modelo
torch.save(model.state_dict(), "retinanet_fracture2.pth")
print("Modelo guardado en retinanet_fracture2.pth")

C:\Users\lenin\AppData\Local\Temp\ipykernel_19560\3862803722.py:97: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
Epoch 1/10:   0%|          | 0/104 [00:00<?, ?batch/s]C:\Users\lenin\AppData\Local\Temp\ipykernel_19560\3862803722.py:112: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1/10: 100%|██████████| 104/104 [00:46<00:00,  2.24batch/s, loss=1.0943]


Epoch 1/10, Loss: 1.1795


Epoch 2/10: 100%|██████████| 104/104 [00:43<00:00,  2.40batch/s, loss=1.0811]


Epoch 2/10, Loss: 1.1386


Epoch 3/10: 100%|██████████| 104/104 [00:43<00:00,  2.37batch/s, loss=1.1351]


...

Epoch 8/10, Loss: 1.1144


Epoch 9/10: 100%|██████████| 104/104 [00:38<00:00,  2.72batch/s, loss=1.0704]


Epoch 9/10, Loss: 1.1133


Epoch 10/10: 100%|██████████| 104/104 [00:38<00:00,  2.67batch/s, loss=1.1398]


Epoch 10/10, Loss: 1.1106
Modelo guardado en retinanet_fracture2.pth


In [54]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Carga el modelo entrenado
model_loaded = retinanet_resnet50_fpn(pretrained=True)

# Asegura que la capa cls_logits tiene las mismas dimensiones antes de cargar
model_loaded.head.classification_head.cls_logits = torch.nn.Conv2d(
    out_channels, num_anchors * num_classes, kernel_size=3, stride=1, padding=1
)
model_loaded.load_state_dict(torch.load("retinanet_fracture2.pth", map_location=torch.device('cpu')))
model_loaded.eval()


# Prepara el conjunto de validación
images_path = r"C:\Users\lenin\Documents\Universidad_2024-2025\VC\Trabajo_final\Pruebas2\bone2_coco\valid"
annotations_file = os.path.join(images_path, '_annotations.json')

dataset_val = FractureDataset(images_path, annotations_file, transforms=transforms)
dataloader_val = DataLoader(dataset_val, batch_size=8, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

# Inicializa la métrica para calcular el mAP
metric = MeanAveragePrecision()

# Realiza la predicción y calcula las métricas
with torch.no_grad():  
    for images, targets in dataloader_val:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Realiza las predicciones
        predictions = model(images)

        # Actualiza la métrica con las predicciones y los valores reales
        metric.update(predictions, targets)

# Calcula las métricas
metrics = metric.compute()

# Almacena los resultados
results = []
results.append({
    "model": "RetinaNet",
    "precision": metrics["map_50"],  # mAP en IoU=0.5
    "recall": metrics["mar_100"],    # Recall con 100 detecciones
    "mAP50": metrics["map_50"],      # mAP en IoU=0.5
    "mAP50-95": metrics["map"]       # mAP en IoU=0.5:0.95
})

print(results)

[{'model': 'RetinaNet', 'precision': tensor(0.0125), 'recall': tensor(0.1733), 'mAP50': tensor(0.0125), 'mAP50-95': tensor(0.0035)}]


In [55]:
# 1. Crea el Dataset personalizado
class FractureDataset(Dataset):
    def __init__(self, images_path, annotations_file, transforms=None):
        self.images_path = images_path
        with open(annotations_file, 'r') as f:
            data = json.load(f)
        
        self.images_info = {img['id']: img for img in data['images']}
        self.annotations = self.process_annotations(data['annotations'])
        self.transforms = transforms

    def process_annotations(self, annotations):
        processed = {}
        for ann in annotations:
            img_id = ann['image_id']
            if img_id not in processed:
                processed[img_id] = {'boxes': [], 'labels': []}
            bbox = ann['bbox']
            x_min, y_min, width, height = bbox
            x_max = x_min + width
            y_max = y_min + height
            processed[img_id]['boxes'].append([x_min, y_min, x_max, y_max])
            processed[img_id]['labels'].append(1)  # Clase "fracture" siempre es 1
        return processed

    def __len__(self):
        return len(self.images_info)

    def __getitem__(self, idx):
        img_id = list(self.images_info.keys())[idx]
        img_info = self.images_info[img_id]
        img_path = os.path.join(self.images_path, img_info['file_name'])
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Obtiene las anotaciones
        if img_id in self.annotations:
            annotations = self.annotations[img_id]
            boxes = torch.tensor(annotations['boxes'], dtype=torch.float32)
            labels = torch.tensor(annotations['labels'], dtype=torch.int64)
        else:
            boxes = torch.empty((0, 4), dtype=torch.float32)
            labels = torch.empty((0,), dtype=torch.int64)

        target = {
            'boxes': boxes,
            'labels': labels,
            'image_id': torch.tensor([idx])
        }

        # Aplicar transformaciones
        if self.transforms:
            img = self.transforms(img)

        return img, target

# 2. Define transformaciones
class ResizeTransform:
    def __call__(self, image):
        image = cv2.resize(image, (640, 640)) 
        return F.to_tensor(image)

transforms = ResizeTransform()

# 3. Inicializa el dataset y dataloaders
images_path = r"C:\Users\lenin\Documents\Universidad_2024-2025\VC\Trabajo_final\Pruebas3\bone3_coco\train"
annotations_file = os.path.join(images_path, '_annotations.json')

dataset = FractureDataset(images_path, annotations_file, transforms=transforms)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

# 4. Carga RetinaNet preentrenado
model = retinanet_resnet50_fpn(pretrained=True)

num_classes = 2

out_channels = model.head.classification_head.conv[0].out_channels 
num_anchors = model.head.classification_head.num_anchors
model.head.classification_head.num_classes = num_classes

cls_logits = torch.nn.Conv2d(out_channels, num_anchors * num_classes, kernel_size=3, stride=1, padding=1)
torch.nn.init.normal_(cls_logits.weight, std=0.01)
torch.nn.init.constant_(cls_logits.bias, -math.log((1 - 0.01) / 0.01))

model.head.classification_head.cls_logits = cls_logits

# Congela todas las capas excepto la última
for param in model.parameters():
    param.requires_grad = False

# Activa gradiente solo para la última capa (cls_logits)
for param in model.head.classification_head.cls_logits.parameters():
    param.requires_grad = True

# 5. Configura el optimizador
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001, weight_decay=0.0001)
scaler = GradScaler()

# 6. Define la función de entrenamiento
def train_one_epoch(model, optimizer, dataloader, device, epoch, num_epochs):
    model.train()
    epoch_loss = 0

    with tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as pbar:
        for images, targets in pbar:
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            optimizer.zero_grad()

            try:
                with autocast():
                    loss_dict = model(images, targets)
                    if not loss_dict:
                        print("Lote vacío, saltando...")
                        continue

                    classification_logits = loss_dict.get('classification', torch.tensor(0.0, device=device))
                    bbox_regression = loss_dict.get('bbox_regression', torch.tensor(0.0, device=device))

                    # Verificar tensores vacíos
                    if classification_logits.numel() == 0 or bbox_regression.numel() == 0:
                        print("Tensor vacío en pérdidas, saltando lote...")
                        continue

                    # Cálculo de la pérdida con sigmoide focal
                    classification_loss = sigmoid_focal_loss(
                        classification_logits,
                        torch.zeros_like(classification_logits),
                        reduction="sum"
                    )

                    bbox_loss = bbox_regression.sum()

                    losses = classification_loss + bbox_loss

                    # Verificar valores numéricamente inestables
                    if not torch.isfinite(losses).all():
                        print("Pérdidas NaN o Inf, restableciendo gradientes y saltando lote...")
                        optimizer.zero_grad()
                        scaler.update()
                        continue

                # Backpropagación y optimización
                scaler.scale(losses).backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
                scaler.step(optimizer)
                scaler.update()

                epoch_loss += losses.item()
                pbar.set_postfix({"loss": f"{losses.item():.4f}"})

            except Exception as e:
                print(f"Error en el lote: {e}")
                continue

    return epoch_loss / len(dataloader)

# 7. Entrena el modelo
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# Cambiar a modo de evaluación para pruebas
def evaluate_model(model, dataloader, device):
    model.eval()  # Modo de evaluación
    results = []
    with torch.no_grad():  # Desactiva gradientes
        for images, targets in dataloader:
            images = [img.to(device) for img in images]
            outputs = model(images)
            results.extend(outputs)
    return results

num_epochs = 10
for epoch in range(num_epochs):
    epoch_loss = train_one_epoch(model, optimizer, dataloader, device, epoch, num_epochs)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")
    test_dataloader = DataLoader(dataset, batch_size=8, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))
    evaluation_results = evaluate_model(model, test_dataloader, device)

# 8. Guarda el modelo
torch.save(model.state_dict(), "retinanet_fracture3.pth")
print("Modelo guardado en retinanet_fracture3.pth")

C:\Users\lenin\AppData\Local\Temp\ipykernel_19560\201642441.py:97: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
Epoch 1/10:   0%|          | 0/218 [00:00<?, ?batch/s]C:\Users\lenin\AppData\Local\Temp\ipykernel_19560\201642441.py:112: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1/10: 100%|██████████| 218/218 [01:37<00:00,  2.25batch/s, loss=1.0943]


Epoch 1/10, Loss: 1.0702


Epoch 2/10: 100%|██████████| 218/218 [01:19<00:00,  2.73batch/s, loss=0.9885]


Epoch 2/10, Loss: 1.0386


Epoch 3/10: 100%|██████████| 218/218 [01:18<00:00,  2.77batch/s, loss=0.9720]


...

Epoch 8/10, Loss: 1.0085


Epoch 9/10: 100%|██████████| 218/218 [01:21<00:00,  2.68batch/s, loss=0.9863]


Epoch 9/10, Loss: 1.0067


Epoch 10/10: 100%|██████████| 218/218 [01:18<00:00,  2.79batch/s, loss=0.9748]


Epoch 10/10, Loss: 1.0053
Modelo guardado en retinanet_fracture3.pth


In [57]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Carga el modelo entrenado
model_loaded = retinanet_resnet50_fpn(pretrained=True)

# Asegura que la capa cls_logits tiene las mismas dimensiones antes de cargar
model_loaded.head.classification_head.cls_logits = torch.nn.Conv2d(
    out_channels, num_anchors * num_classes, kernel_size=3, stride=1, padding=1
)
model_loaded.load_state_dict(torch.load("retinanet_fracture3.pth", map_location=torch.device('cpu')))
model_loaded.eval()


# Prepara el conjunto de validación
images_path = r"C:\Users\lenin\Documents\Universidad_2024-2025\VC\Trabajo_final\Pruebas3\bone3_coco\valid"
annotations_file = os.path.join(images_path, '_annotations.json')

dataset_val = FractureDataset(images_path, annotations_file, transforms=transforms)
dataloader_val = DataLoader(dataset_val, batch_size=8, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

# Inicializa la métrica para calcular el mAP
metric = MeanAveragePrecision()

# Realiza la predicción y calcula las métricas
with torch.no_grad():  
    for images, targets in dataloader_val:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Realiza las predicciones
        predictions = model(images)

        # Actualiza la métrica con las predicciones y los valores reales
        metric.update(predictions, targets)

# Calcula las métricas
metrics = metric.compute()

# Almacena los resultados
results = []
results.append({
    "model": "RetinaNet",
    "precision": metrics["map_50"],  # mAP en IoU=0.5
    "recall": metrics["mar_100"],    # Recall con 100 detecciones
    "mAP50": metrics["map_50"],      # mAP en IoU=0.5
    "mAP50-95": metrics["map"]       # mAP en IoU=0.5:0.95
})

print(results)

[{'model': 'RetinaNet', 'precision': tensor(0.1355), 'recall': tensor(0.3471), 'mAP50': tensor(0.1355), 'mAP50-95': tensor(0.0367)}]
